<a href="https://colab.research.google.com/github/NASA-Hackathon-Imaginarium-Team/AI-Team/blob/main/lightgbm_K2_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from lightgbm import LGBMClassifier
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from google.colab import files
import io
import time
from lightgbm import LGBMClassifier
import numpy as np
import lightgbm as lgb

print("Upload your CSV file...")
uploaded = files.upload()
file_name = next(iter(uploaded))  # get uploaded file name


Upload your CSV file...


StopIteration: 

In [ ]:

start_time = time.time()
# Step 2: Load dataset
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

label_encoder = LabelEncoder()
df_encoded = pd.get_dummies(df, columns=['discoverymethod', 'disc_facility'], drop_first=True)

y = label_encoder.fit_transform(df_encoded['disposition'])

# Step 4: Features = all columns except target
X = df_encoded.drop(columns=['disposition'])

# split into train and test only
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Clean feature names for LightGBM
X_train.columns = (
    X_train.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
)
X_test.columns = (
    X_test.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
)

lgb_model = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=3,          # number of unique target classes
    boosting_type='gbdt',
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

lgb_model.fit(X_train, y_train)

# predict probabilities on test set
y_pred = lgb_model.predict_proba(X_test)

from sklearn.metrics import accuracy_score

# predict labels
y_label_pred = lgb_model.predict(X_test)

# compute accuracy
accuracy = accuracy_score(y_test, y_label_pred)
print("Accuracy:", accuracy * 100)

end_time = time.time()
print(f"\nExecution time: {end_time - start_time:.2f} seconds")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Define the base model
num_classes = len(np.unique(y_train))
lgb_model = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=num_classes,
    boosting_type='gbdt',
    random_state=42
)

# Define the parameter search space
param_dist = {
    'num_leaves': randint(20, 150),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'n_estimators': randint(50, 300),
    'min_child_samples': randint(10, 100),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
}

# Run randomized hyperparameter search
random_search = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=param_dist,
    n_iter=30,              # try 30 random combinations (increase for better tuning)
    scoring='accuracy',
    cv=3,                   # 3-fold cross-validation
    verbose=1,
    n_jobs=-1,
    random_state=42
)

print("Running hyperparameter search...")
random_search.fit(X_train, y_train)

# Get the best model
print("\nBest hyperparameters found:")
print(random_search.best_params_)

best_model = random_search.best_estimator_


In [ ]:
# Use the optimized parameters
best_params = {
    'colsample_bytree': 0.9077307142274171,
    'learning_rate': 0.22205720315428515,
    'max_depth': 5,
    'min_child_samples': 44,
    'n_estimators': 82,
    'num_leaves': 142,
    'reg_alpha': 0.07404465173409036,
    'reg_lambda': 0.3584657285442726,
    'subsample': 0.5579345297625649
}

# Initialize LightGBM model with best parameters
lgb_model = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=3,
    boosting_type='gbdt',
    random_state=42,
    **best_params
)

# Train the tuned model
lgb_model.fit(X_train, y_train)

# Predict on test set
y_pred = lgb_model.predict_proba(X_test)
y_label_pred = lgb_model.predict(X_test)

# Evaluate performance
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_label_pred)
print(f"Tuned Model Accuracy: {accuracy * 100:.2f}%\n")

print("Classification Report:")
print(classification_report(y_test, y_label_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_label_pred))


In [ ]:
import pickle
from google.colab import files

# Save trained model as pickle file
model_filename = "lightgbm_tuned_model.pkl"

with open(model_filename, "wb") as file:
    pickle.dump(lgb_model, file)

print(f"Model saved successfully as {model_filename}")

# Download the file
files.download(model_filename)



In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np
import re

# Clean feature names
X.columns = [
    re.sub(r'[^A-Za-z0-9_]+', '_', col) for col in X.columns
]

# Define stratified k-fold CV (keeps class proportions)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(
    lgb_model,
    X, y,
    cv=cv,
    scoring='accuracy',
    n_jobs=-1
)

# Display results
print("✅ Cross-validation scores:", cv_scores)
print("📊 Mean CV accuracy:", np.mean(cv_scores))
print("📉 Standard deviation:", np.std(cv_scores))


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, log_loss

# Predictions
y_train_pred = lgb_model.predict(X_train)
y_test_pred = lgb_model.predict(X_test)

# Accuracy
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Optional: log loss for multiclass probability predictions
train_proba = lgb_model.predict_proba(X_train)
test_proba = lgb_model.predict_proba(X_test)
print(f"Training Log Loss: {log_loss(y_train, train_proba):.4f}")
print(f"Test Log Loss: {log_loss(y_test, test_proba):.4f}")
